<a href="https://colab.research.google.com/github/Suhas-S63/Deep_Learning_Lab/blob/main/2348563_DL_Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [16]:
! kaggle datasets download nih-chest-xrays/sample --force

100% 4.19G/4.20G [02:25<00:00, 33.8MB/s]
100% 4.20G/4.20G [02:25<00:00, 30.9MB/s]


In [17]:
! kaggle datasets download tawsifurrahman/tuberculosis-tb-chest-xray-dataset --force

 99% 660M/663M [00:25<00:00, 20.7MB/s]
100% 663M/663M [00:25<00:00, 27.0MB/s]


In [18]:
!unzip -q /content/sample.zip -d /content/sample

replace /content/sample/sample/sample/images/00000013_005.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [60]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Lambda
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [21]:
image_dir = '/content/sample/sample/images'
total_images = 0
for file in os.listdir(image_dir):
    if file.endswith('.png') or file.endswith('.jpg') or file.endswith('.jpeg'):
        total_images += 1
print("Total number of images:", total_images)


Total number of images: 5606


In [22]:
# Deleting some images

image_dir = '/content/sample/sample/images'
#  list of all image files
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg')]

num_images = min(3606, len(image_files))

# Iterate through the first 'num_images' images and delete them
for i in range(num_images):
    image_path = os.path.join(image_dir, image_files[i])
    os.remove(image_path)

print(f"{num_images} images deleted from {image_dir}")

3606 images deleted from /content/sample/sample/images


In [40]:
image_dir = '/content/sample/sample/images'
total_images = 0
for file in os.listdir(image_dir):
    if file.endswith('.png') or file.endswith('.jpg') or file.endswith('.jpeg'):
        total_images += 1
print("Total number of images:", total_images)

Total number of images: 2000


In [42]:
image_dir = '/content/sample/sample/images'

# labels file
csv_file = '/content/sample/sample/sample_labels.csv'

df = pd.read_csv(csv_file)

# deleting the first 3803 records
df = df.iloc[3606:]

# Saving
df.to_csv('/content/sample_labels_updated.csv', index=False)

target_size = (224, 224)
images = []
labels = []

# Filter out the filenames present in both the directory and the DataFrame
valid_filenames = set(os.listdir(image_dir)) & set(df['Image Index'])

for filename in valid_filenames:
    img_path = os.path.join(image_dir, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # image as grayscale
    if img is not None:
        img_resized = cv2.resize(img, target_size)
        img_normalized = img_resized / 255.0
        images.append(img_normalized)
        label = df[df['Image Index'] == filename]['Finding Labels'].values
        if len(label) > 0:  # check for label emptiness
            labels.append(label[0])
        else:
            print(f"No label found for image: {filename}")
    else:
        print(f"Failed to load image: {img_path}")

images = np.array(images)
labels = np.array(labels)

# Split dataset
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Reshape grayscale images to include channel dimension
x_train = np.expand_dims(x_train, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(x_train)

In [67]:
data = pd.read_csv('/content/sample_labels_updated.csv')
data.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00017736_006.png,No Finding,6,17736,036Y,M,PA,2694,2709,0.143,0.143
1,00017747_002.png,Emphysema,2,17747,033Y,M,PA,2992,2991,0.143,0.143
2,00017747_006.png,No Finding,6,17747,033Y,M,PA,2992,2991,0.143,0.143
3,00017747_019.png,No Finding,19,17747,033Y,M,PA,2992,2991,0.143,0.143
4,00017747_031.png,Nodule,31,17747,034Y,M,AP,3056,2544,0.139,0.139


## ResNet Model

In [54]:
# ResNet model
res = Sequential()
res.add(Lambda(lambda x: x * tf.ones_like(x), input_shape=(224, 224, 3)))  # Duplicate grayscale channel
res.add(ResNet50(include_top=False, input_shape=(224, 224, 3), pooling='max', weights='imagenet'))
res.add(Dense(512, activation='relu'))
res.add(Dropout(0.5))
res.add(Dense(1, activation='softmax'))

In [63]:
res.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Custom Model

In [61]:
# Custom CNN model
custom = Sequential()
custom.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
custom.add(BatchNormalization())
custom.add(MaxPooling2D((2, 2)))
custom.add(Conv2D(64, (3, 3), activation='relu'))
custom.add(BatchNormalization())
custom.add(MaxPooling2D((2, 2)))
custom.add(Conv2D(128, (3, 3), activation='relu'))
custom.add(BatchNormalization())
custom.add(MaxPooling2D((2, 2)))
custom.add(Flatten())
custom.add(Dense(512, activation='relu'))
custom.add(Dropout(0.5))
custom.add(Dense(1, activation='softmax'))

In [64]:
custom.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [57]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [65]:
# ResNet model training
resnet_training = res.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=5,
                                  validation_data=(x_val, y_val), callbacks=[early_stop])

Epoch 1/5


UnimplementedError: Graph execution error:

Detected at node Cast_1 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-packages

In [ ]:
# Custom CNN model training
custom_training = custom.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=5,
                                          validation_data=(x_val, y_val), callbacks=[early_stop])